In [ ]:
#r "nuget:Blockfrost.Api,0.0.5-alpha"

Installed Packages Blockfrost.Api, 0.0.5-alpha

# blockfrost-dotnet

## Usage

Using the SDK is pretty straight-forward as you can see from the following example.

In [ ]:
using System.IO;
using Blockfrost.Api;
using Blockfrost.Api.Extensions;
using Microsoft.Extensions.DependencyInjection;
using System.Text.Json;


var apiKey = "YOUR_BLOCKFROST_PROJECT_ID";
var network = "NETWORK_OF_THE_PROJECT_ID"; 
var sender_address = "SENDER_ADDR";
var receiver_address = "RECEIVER_ADDR";
var signedTx = File.ReadAllText("path/to/your/signed/transaction");

var provider = new ServiceCollection().AddBlockfrost(network, apiKey).BuildServiceProvider();

var blockService = provider.GetRequiredService<IBlockService>();
var addressService = provider.GetRequiredService<IAddressService>();
var transactionService = provider.GetRequiredService<ITransactionService>();

((ABlockfrostService)transactionService).ReadResponseAsString = true;

int total = 0;
int? slot = 0;

Console.WriteLine($"Summing UTxO Amounts for {sender_address}...");
var utxoSender = await addressService.UtxosAllAsync(sender_address,100,0,ESortOrder.Asc).ConfigureAwait(false);
int totalSender = utxoSender.Sum(m => m.Amount.Sum(a => int.Parse(a.Quantity)));
System.Console.WriteLine($"Sender Total: {totalSender} lovelace");

Console.WriteLine($"Summing UTxO Amounts for {receiver_address}...");
var utxoReceiver = await addressService.UtxosAllAsync(receiver_address,100,0,ESortOrder.Asc).ConfigureAwait(false);
int totalReceiver = utxoReceiver.Sum(m => m.Amount.Sum(a => int.Parse(a.Quantity)));
System.Console.WriteLine($"Receiver Total: {totalReceiver} lovelace");

var tip = await blockService.GetLatestBlockAsync();
slot = tip.Slot;

Console.WriteLine($"Tip now at Epoch {tip.Epoch} Slot {tip.Slot} Block {tip.Height}");
await Task.Delay(TimeSpan.FromSeconds(1));

Console.WriteLine("Sending transaction in 3... 2... 1...");
await Task.Delay(TimeSpan.FromSeconds(3));

Console.WriteLine(signedTx);
var txid = await transactionService.SubmitAsync(signedTx);

Console.WriteLine($"");
Console.WriteLine($"************************************************************************************************************************");
Console.WriteLine($"                              Your Transaction was transmitted to the {network}");
Console.WriteLine($" You can wait for it on the {network} explorer: https://explorer.cardano-{network}.iohkdev.io/en/transaction?id={txid}");
Console.WriteLine($"                    Or wait here and hope your transaction makes it into the next block...");
Console.WriteLine($"");
Console.WriteLine($"      If traffic on the {network} is slow, it can take some time until the next block is ready... Try to be patient.");
Console.WriteLine($"************************************************************************************************************************");

while(slot == tip.Slot)
{
    Console.WriteLine("Waiting for next block...");
    await Task.Delay(TimeSpan.FromSeconds(3));
    tip = await blockService.GetLatestBlockAsync();
}

Console.WriteLine($"Tip now at Epoch {tip.Epoch} Slot {tip.Slot} Block {tip.Height}");
totalSender = utxoSender.Sum(m => m.Amount.Sum(a => int.Parse(a.Quantity)));
System.Console.WriteLine($"Sender Total: {totalSender} lovelace");

totalReceiver = utxoReceiver.Sum(m => m.Amount.Sum(a => int.Parse(a.Quantity)));
System.Console.WriteLine($"Receiver Total: {totalReceiver} lovelace");

Console.WriteLine($"Have a very lovelacey day...");
                    


Summing UTxO Amounts for addr_test1vrgvgwfx4xyu3r2sf8nphh4l92y84jsslg5yhyr8xul29rczf3alu...
Sender Total: 988258310 lovelace
Summing UTxO Amounts for addr_test1vqah2xrfp8qjp2tldu8wdq38q8c8tegnduae5zrqff3aeec7g467q...
Receiver Total: 10000000 lovelace
Querying tip...
Tip now at Epoch 152 Slot 35399692 Block 2855047
Sending transaction in...
3...
2...
1...
{
    "type": "Tx MaryEra",
    "description": "",
    "cborHex": "83a3008182582002ffae36f91deee20a784e48ef60d604c20c983d84eb4cccbcbb19e6a315f20900018282581d60d0c43926a989c88d5049e61bdebf2a887aca10fa284b9067373ea28f1a3ad5b56d82581d603b75186909c120a97f6f0ee6822701f075e5136f3b9a08604a63dce71a000f4240021a0002a859a100818258200bc7720123b3fd7bc8c0573bbd7df6577cef1a3385ab79959d1319d373f5ebe15840620fa1206cf5634fba738ca1ea76c8b25018cd3755af8c67e7f9c219aaa07ac7a68133e97e68209e08fd4eaa44076ce602b6af1f8c14003ce7b54b487197c40df6"
}


****
Your Transaction was transmitted to the testnet

You can wait for it on the testnet explorer: https://explorer.